# Project - Day 4 - MLFlow evaluate and fit

## Set parameters
The cell below has been already tagged as `parameters`. So use it to include any papermill parameter you think it would be useful to change from at MLFlow runtime. (e.g. the location of models trained in the previous step)

In [ ]:
model_run_uri = "dummy"

## Loading libraries, data and model

### Loading libraries and model from MLFlow

In [1]:
import warnings

warnings.filterwarnings('ignore')
## We will be using Numpy, Pyplot and Tensorflow as our scientific tool box
import numpy as np 
import matplotlib.pyplot as plt
import tensorflow as tf

## BytesIO for defining in-memory file-like objects
from io import BytesIO

## Dask and in particular dask array for defining OOM pipelines
import dask
import dask.array as da

## Progress bars
from tqdm import trange

import mlflow


2023-10-16 09:57:23.042088: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/opt/conda/lib/python3.10/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'dciangot-mlflow.131.154.99.220.myip.cloud.infn.it'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


### Reproduce the final result plot based on the new model trained from the pipeline

You should now be able to reproduce the steps of the Day-3 model deployment and adapt it to the MLFlow pipeline:

- load the model from the artifact location of the previous step
  - little help: `mlflow.artifacts.download_artifacts(artifact_uri=model_run_uri, dst_path="./models")`
- evaluate and fit the results, storing the plot as MLFlow artifacts


In [ ]:
from io import BytesIO
import boto3
import hashlib
import os


username = "adavanzo"
hash_object = hashlib.md5(f'{username}'.encode())
password = hash_object.hexdigest()

s3 = boto3.resource(
    's3',
    endpoint_url = "https://minio.131.154.99.220.myip.cloud.infn.it",
    aws_access_key_id=username,
    aws_secret_access_key=password
)

In [ ]:
s3client = boto3.client('s3',
    aws_access_key_id=username,
    aws_secret_access_key=password,
    endpoint_url="https://minio.131.154.99.220.myip.cloud.infn.it",
    region_name='default',)

objects_name = []
bucket_name = 'adavanzo'
resp = s3client.list_objects(Bucket=bucket_name)
for object in resp['Contents']:
        objects_name.append(object['Key'])

In [ ]:
objects_name = objects_name[-80:]

In [ ]:
def load_npz_from_minio(s3, object_name):
    obj_npz = s3.Bucket(username).Object(object_name)
    bytes = BytesIO(
        obj_npz.get()["Body"].read()
        )
    np_file = np.load(bytes)
    return np_file

In [ ]:
@dask.delayed
def load_array_from_minio(minio_client, object_name, npz_key):
    """Load an array identified by npz_key from an npz file in Minio"""
    npz = load_npz_from_minio(minio_client, object_name)
    return npz[npz_key]


delayed_images = [
    da.from_delayed(
        load_array_from_minio(s3, obj, 'image'),
        shape=(200, 128, 128),
        dtype=np.float64
    )
    for obj in objects_name
]

delayed_timestamps = [
    da.from_delayed(
        load_array_from_minio(s3, obj, 'tstamp'),
        shape=(200, ),
        dtype=np.datetime64
    )
    for obj in objects_name
]

images = da.concatenate(delayed_images)
tstamps = da.concatenate(delayed_timestamps)

In [ ]:
batch_size = 10

#images.compute()
X_test = da.rechunk(images, chunks = (batch_size, 128, 128))
#X_test[-15:].compute()

predictions = np.concatenate([classifier.predict_on_batch(X) for X in X_test.blocks])

In [ ]:
with mlflow.start_run(tags={"mlflow.runName": "Test"}) as mlrun:
    local_model_path = mlflow.artifacts.download_artifacts(
        artifact_uri=model_path, dst_path="./models"
    )
    loaded_model = mlflow.pyfunc.load_model(local_model_path)
    predictions = np.concatenate([classifier.predict(X) for X in X_test.blocks])

In [ ]:
def plot_histogram(predictions):
    """Makes a histogram of the CNN predictions for the CYGNO-SIM acquired data"""
    plt.hist(predictions, bins=np.linspace(0, 1, 51), label="CYGNO-SIM")
    plt.yscale('log')
    plt.xlabel("Response of the CNN")
    plt.ylabel("Number of acquired events")
    plt.legend()
    plt.show()